# Carga de datos 


## Librerias 

In [1]:
import sys
sys.path.append('../')
# Importación de las funciones creadas en nuestro archivo de soporte
# -----------------------------------------------------------------------

from src import soporte_queries_creacion as sqc
from src import soporte_queries_insercion as sqi
from src import creacion_bbdd as cb


# Para trabajar con los DataFrames
# -----------------------------------------------------------------------
import pandas as pd
# Para trabajar con postgresql
# -----------------------------------------------------------------------
import psycopg2

# Para trabajar gestionar los nulos
# -----------------------------------------------------------------------
import numpy as np

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Ignorar warings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

## Creacionde base de datos

In [2]:
def establecer_conn(database_name, postgres_pass, usuario, host="localhost"):
    """
    Establece una conexión a una base de datos de PostgreSQL.

    Params:
        - database_name (str): El nombre de la base de datos a la que conectarse.
        - postgres_pass (str): La contraseña del usuario de PostgreSQL.
        - usuario (str): El nombre del usuario de PostgreSQL.
        - host (str, opcional): La dirección del servidor PostgreSQL. Por defecto es "localhost".

    Returns:
        psycopg2.extensions.connection: La conexión establecida a la base de datos PostgreSQL.

    """

    # Crear la conexión a la base de datos PostgreSQL
    conn = psycopg2.connect(
        host=host,
        user=usuario,
        password=postgres_pass,
        database=database_name
    )

    # Establecer la conexión en modo autocommit
    conn.autocommit = True # No hace necesario el uso del commit al final de cada sentencia de insert, delete, etc.
    
    return conn



# conexion a postgres
conn = establecer_conn("postgres", "admin", "postgres") # Nos conectamos a la base de datos de postgres por defecto para poder crear la nueva base de datos

# creamos un cursor con la conexion que acabamos de crear
cur = conn.cursor()


In [3]:
def crear_db(database_name):
    # conexion a postgres
    conn = establecer_conn("postgres", "admin", "postgres") # Nos conectamos a la base de datos de postgres por defecto para poder crear la nueva base de datos
    
    # creamos un cursor con la conexion que acabamos de crear
    cur = conn.cursor()
    
    cur.execute("SELECT 1 FROM pg_database WHERE datname = %s", (database_name,))
    
    # Almacenamos en una variable el resultado del fetchone. Si existe tendrá una fila sino será None
    bbdd_existe = cur.fetchone()
    
    # Si bbdd_existe es None, creamos la base de datos
    if not bbdd_existe:
        cur.execute(f"CREATE DATABASE {database_name};")
        print(f"Base de datos {database_name} creada con éxito")
    else:
        print(f"La base de datos ya existe")
        
    # Cerramos el cursor y la conexion
    cur.close()
    conn.close()


## Creacionde base de Tablas


In [4]:
crear_db("hospitales")

La base de datos ya existe


In [5]:
conexion = establecer_conn("hospitales", "admin", "postgres")

conexion.cursor().execute(sqc.query_creation_hospitales)

In [6]:
conexion.cursor().execute(sqc.query_creation_tipo_hosp)
conexion.cursor().execute(sqc.query_creation_gastos)
conexion.cursor().execute(sqc.query_creation_ingresos)

In [ ]:
df_hospi = pd.read_csv("../datos/df_ncodi.csv", index_col=0)
lista_tuplas = [tuple([int(fila)]) for fila in df_hospi.values]
cb.dbeaver_commitmany(conexion, sqi.query_carga_hospitales, lista_tuplas)

In [ ]:
df_hospi = pd.read_csv("../datos/df_tipo_hospitalizacion.csv", index_col=0)
lista_tuplas = [tuple(fila) for fila in df_hospi.values]
cb.dbeaver_commitmany(conexion, sqi.query_carga_tipo_hospi, lista_tuplas)

In [ ]:
df_ingresos = pd.read_csv("../datos/df_ingresos_final.csv", index_col=0)
lista_tuplas = [tuple(fila) for fila in df_ingresos.values]
tuplas_float = [tuple(float(valor) for valor in tupla) for tupla in lista_tuplas]

cb.dbeaver_commitmany(conexion, sqi.query_carga_ingresos, tuplas_float)

In [30]:
df_gastos = pd.read_csv("../datos/df_gastos_final.csv", index_col=0)
lista_tuplas = [tuple(fila) for fila in df_gastos.values]
tuplas_float = [tuple(float(valor) for valor in tupla) for tupla in lista_tuplas]

cb.dbeaver_commitmany(conexion, sqi.query_carga_gastos, tuplas_float)

NumericValueOutOfRange: entero fuera de rango


In [31]:
df_gastos.dtypes

año                    int64
totalcompra            int64
producfarma          float64
materialsani           int64
implantes            float64
restomateriasani     float64
servcontratado       float64
trabajocontratado    float64
xrestocompras        float64
variaexistencias     float64
servexteriores         int64
sumistro             float64
xrestoserviexter     float64
gastopersonal          int64
sueldos                int64
indemnizacion        float64
segsocempresa          int64
otrgassocial         float64
dotaamortizacion     float64
perdidadeterioro     float64
xrestogasto          float64
totcompragasto         int64
ncodi                float64
dtype: object